# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,42.06,89,100,25.70,TF,05/06/2024
1,1,dakhla,23.6848,-15.9580,71.62,73,89,26.46,EH,05/06/2024
2,2,grytviken,-54.2811,-36.5092,33.75,82,96,10.18,GS,05/06/2024
3,3,adamstown,-25.0660,-130.1015,71.80,77,61,21.43,PN,05/06/2024
4,4,taybad,34.7400,60.7756,78.44,23,45,4.45,IR,05/06/2024


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    tiles = "EsriImagery", 
    size = "Humidity", 
    scale = 0.7,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[
    (city_data_df['Max Temp'] < 85) & 
    (city_data_df['Max Temp'] > 70) & 
    (city_data_df['Wind Speed'] < 10) & 
    (city_data_df['Cloudiness'] < 10) & 
    (city_data_df['Humidity'] < 80) & 
    (city_data_df['Humidity'] > 50)
]

# Drop any rows with null values
columns_to_check = ['Max Temp', 'Wind Speed', 'Cloudiness', 'Humidity']
ideal_cities_df = ideal_cities_df.dropna(subset=columns_to_check)

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
70,70,nova vicosa,-17.8919,-39.3719,82.58,63,3,9.19,BR,05/06/2024
187,187,ilha de mocambique,-15.0342,40.7358,76.21,79,2,5.12,MZ,05/06/2024
202,202,holualoa,19.6228,-155.9522,71.44,79,0,0.00,US,05/06/2024
208,208,mangai,-4.0500,19.5333,78.71,53,1,1.48,CD,05/06/2024
232,232,luau,-10.7073,22.2247,70.32,57,5,1.32,AO,05/06/2024
281,281,bac lieu,9.2850,105.7244,84.85,66,2,8.19,VN,05/06/2024
401,401,marsa alam,25.0757,34.8918,74.91,51,0,7.27,EG,05/06/2024
477,477,athens,37.9795,23.7162,70.90,51,0,5.01,GR,05/06/2024
553,553,toliara,-23.3500,43.6667,71.65,71,0,6.67,MG,05/06/2024
581,581,mukalla,14.5425,49.1242,84.54,78,0,4.36,YE,05/06/2024


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = pd.Series(dtype='object')

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
70,nova vicosa,BR,-17.8919,-39.3719,63,NaN
187,ilha de mocambique,MZ,-15.0342,40.7358,79,NaN
202,holualoa,US,19.6228,-155.9522,79,NaN
208,mangai,CD,-4.0500,19.5333,53,NaN
232,luau,AO,-10.7073,22.2247,57,NaN
281,bac lieu,VN,9.2850,105.7244,66,NaN
401,marsa alam,EG,25.0757,34.8918,51,NaN
477,athens,GR,37.9795,23.7162,51,NaN
553,toliara,MG,-23.3500,43.6667,71,NaN
581,mukalla,YE,14.5425,49.1242,78,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = # YOUR CODE HERE
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Hotel del Real
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Hotel del Real


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)